# Imports

Todo: reduce batches for training with model number 2. 
Check how to solve memory problems with tensorflow-directml
Add feature engineering.
Add normalization.

In [1]:
from load_df import load_df

In [2]:
dfx = load_df("data/train/x_train.csv")
dfy = load_df("data/train/y_train.csv")

# Modules instanciation

In [3]:
CATEGORY_COLUMNS = ["venue","action","trade","side"]
SCALAR_COLUMNS = ["price","bid","ask","bid_size","ask_size","flux"]
INDEX_COLUMN= "obs_id"

In [4]:
from encoder import Encoder
encoder = Encoder(category_colums=CATEGORY_COLUMNS)
dfx = encoder.apply(dfx)
CATEGORY_COLUMNS = encoder.new_columns

In [5]:
from splitter import Splitter
TEST_SIZE = 1000
splitter_x = Splitter(df=dfx,
                      batch_size=20000,
                      test_size=TEST_SIZE,
                      index_column=INDEX_COLUMN)
splitter_y = Splitter(df=dfy,
                      batch_size=20000,
                      test_size=TEST_SIZE,index_column=INDEX_COLUMN)

from feature_engineer import FeatureEngineer
feature_engineer = FeatureEngineer(category_columns=CATEGORY_COLUMNS,
                                   scalar_columns=SCALAR_COLUMNS,
                                   index_column=INDEX_COLUMN)

from preparator import Preparator
preparator = Preparator()


In [6]:

from model import create_model
df_x = splitter_x.get_current_slice()
df_x = feature_engineer.apply(df_x)
df_y = splitter_y.get_current_slice()


0


d:\challenge CFM\challenge CFM\feature_engineer.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['{}_lag1'.format(column)] = df.groupby(self.index_column)[column].shift(1)
d:\challenge CFM\challenge CFM\feature_engineer.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['{}_l

0


d:\challenge CFM\challenge CFM\feature_engineer.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['{}_skew'.format(column)] = df.groupby(self.index_column)[column].transform('skew')
d:\challenge CFM\challenge CFM\feature_engineer.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['{}_min'.format(column)] = df.groupby(self.index_column)[column].transform('min')
d:\challenge CFM\challenge CFM\feature_engineer.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [7]:

trainx_array = preparator.input_df_to_array(df=df_x,
                index_column=INDEX_COLUMN)
trainy_array = preparator.output_df_to_array(df=df_y)


In [8]:
N_TIMESTEPS, N_FEATURES, N_OUTPUTS = trainx_array.shape[1], trainx_array.shape[2], trainy_array.shape[1]
model = create_model(n_timesteps=N_TIMESTEPS,
                     n_features=N_FEATURES,
                     n_outputs=N_OUTPUTS,
                     model_type=0)

# Training

In [9]:
VERBOSE, EPOCHS, BATCH_SIZE = 0, 50, 1000
while not splitter_x.end:
    df_x = splitter_x.get_current_slice()
    df_x = feature_engineer.apply(df_x)
    df_y = splitter_y.get_current_slice()
    
    trainx_array = preparator.input_df_to_array(df=df_x,
                     index_column=INDEX_COLUMN)
    trainy_array = preparator.output_df_to_array(df=df_y)

    array_list = [trainx_array,trainy_array]
    new_arrays = preparator.shuffle_2d_sections(array_list)
    trainx_array, trainy_array = new_arrays[0] , new_arrays[1]
    model.fit(trainx_array,
            trainy_array,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=VERBOSE) 

    splitter_x.shift()
    splitter_y.shift()

0


d:\challenge CFM\challenge CFM\feature_engineer.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['{}_lag1'.format(column)] = df.groupby(self.index_column)[column].shift(1)
d:\challenge CFM\challenge CFM\feature_engineer.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['{}_l

0


In [ ]:
model.save("pretrained_second.keras")

# Prediction

In [ ]:
# testx_df = splitter_x.get_test_df()
import pandas as pd
testx_df = pd.read_csv("data/test/x_test.csv")
testx_df = feature_engineer.apply(testx_df)
# testy_df = splitter_y.get_test_df()


In [ ]:
testx_array = preparator.input_df_to_array(testx_df,index_column=INDEX_COLUMN)
# testy_array = preparator.output_df_to_array(testy_df)

In [ ]:
prediction = model.predict(testx_array)
# _, accuracy = model.evaluate(testx_array, testy_array, batch_size=BATCH_SIZE, verbose=0)

In [ ]:
# print(accuracy)

In [ ]:
# output = [max(enumerate(row), key=lambda x: x[1])[0] for row in prediction]
#import pandas as pd
#output_df = pd.DataFrame({"obs_id":list(df_test["obs_id"].unique()),"eq_code_cat":output})
#output_df.to_csv("data/submission_benchmark.csv",index=False)